In [2]:
from utils import *

In [1]:
STEP = 30

In [3]:
paths = np.load('src/all_paths.npy')
paths[:, 1] = paths[:, 1] / STEP
paths[:, 2] = paths[:, 2] / STEP

ped_time = paths[:,0]
ped_x = paths[:,1]
ped_y = paths[:,2]

Pedestrians' average speed: 35 by x-coordinate and 30 by y-coordinate.

So let's assume pedestrian(s) takes the whole 30x30 block. That minimize our 1920x1080 field to 64x36 on which we will be calculate robot's path

In [3]:
np.random.seed(0)

start = (0, np.random.randint(0, FRAME_HEIGHT))
end = (FRAME_WIDTH, np.random.randint(0, FRAME_HEIGHT))
start_time = 4 * np.random.randint(0, 30000)

In [4]:
rob_x = np.empty(0, dtype=np.int64)
rob_y = np.empty(0, dtype=np.int64)

points = [(553, 63), (672, 600), (915, 997), (1618, 358)]

for start, end in zip(points[:-1], points[1:]):
    number_of_steps = int(max(abs(end[0] - start[0]) / 35, abs(end[1] - start[1]) / 30)) + 1
    rob_x = np.append(rob_x, np.linspace(start[0], end[0], number_of_steps).astype(np.int64))
    rob_y = np.append(rob_y, np.linspace(start[1], end[1], number_of_steps).astype(np.int64))

In [5]:
def find_by_point(time, x, y):
    indices = np.arange(NUMBER_OF_PEDESTRIANS)
        
    path = np.array([time, x, y])
        
    for i in indices:
        df = download_pedestrian(i, True)
        df.insert(0, 'index', df.index)
        
        data = np.array(df)
        
        ind = np.where(np.all(data == path, axis=1))[0]
        
        if len(ind) > 0:
            return (i, df.index.values[ind[0]])
        
    return (-1, -1)

In [6]:
answer = 0
time = start_time

for i in range(len(rob_x)):
    x = rob_x[i]
    y = rob_y[i]
    r = 10
    indices = np.where(ped_time == time)
    x_in_time = ped_x[indices]
    y_in_time = ped_y[indices]
    
    intersect = np.where((x_in_time > x-r)*(x_in_time < x+r)*(y_in_time > y-r)*(y_in_time < y+r))[0]
    answer += intersect.size
    
    if intersect.size > 0:
        ped_index = find_by_point(time, x_in_time[intersect[0]], y_in_time[intersect[0]])
        print(i, time, x, y, ped_index[0])
        
    time += 4

0 39380 553 63 3369
2 39388 567 126 3361
3 39392 574 157 3360


In [7]:
answer

3

In [8]:
ped1 = np.array(download_pedestrian(3369))
ped2 = np.array(download_pedestrian(3361))
ped3 = np.array(download_pedestrian(3360))
rob = np.array([rob_x, rob_y]).T

draw_lines(39500, [rob, ped1, ped2, ped3], ['green', 'red', 'blue', 'yellow'], 'img')

FileNotFoundError: [Errno 2] No such file or directory: 'src/Frame/039500.jpg'